In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
#import tensorflow_io as tfio

from IPython.display import display, Audio
from tensorflow import keras
from scipy import signal
from scipy.io import wavfile

In [2]:
DIREC = 'TRAIN_V2/data_out_2/'

SAMPLING_RATE = 16000

In [ ]:
def paths_and_labels_to_dataset(audio_paths, labels):
    """Constructs a dataset of audios and labels."""
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))

In [3]:
def path_to_audio(path):
    """Reads and decodes an audio file."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio

In [4]:
def audio_to_fft(audio):
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64))
    fft = tf.expand_dims(fft, axis=-1)
    #return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])
    return fft

In [5]:
#read the files in train_files
train_files = os.listdir(DIREC + str(18) + '/')
for i in range(len(train_files)):
    sample_rate, samples = wavfile.read(DIREC + str(18) + '/' + train_files[i])
    audio = path_to_audio(DIREC + str(18) + '/' + train_files[i])
    audio_fft = audio_to_fft(audio)
    audio_fft = tf.cast(audio_fft, tf.float32)
    audio_slice = audio[100:]
    audio_tensor = tf.squeeze(audio_slice, axis=[-1])
    #audio = tf.squeeze(audio, axis=-1)
    display(Audio(audio, rate=SAMPLING_RATE))
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

In [6]:
type(audio_fft)

tensorflow.python.framework.ops.EagerTensor